In [4]:
# Importar librerías necesarias
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar datos del consumo de agua desde el archivo Parquet
data_path = '/Users/mgar/Code/datadonsaloocupacio/data/Mostra Set de dades 1_ Consum total agregat.parquet'
df_water = pd.read_parquet(data_path)

# Cargar datos de distritos desde el CSV
districts_path = '/Users/mgar/Code/datadonsaloocupacio/miguel/districtes.csv'
df_districts = pd.read_csv(districts_path)

# Renombrar columnas para facilitar el manejo (basado en los datos proporcionados)
df_water.columns = ['Census_section', 'District', 'Municipality', 'Date', 'Use', 'Number_of_meters', 'Accumulated_consumption_L_per_day']

# Convertir la columna Date a datetime
df_water['Date'] = pd.to_datetime(df_water['Date'])

# Unir con datos de distritos para obtener nombres
df_water['District'] = df_water['District'].astype(int)
df_districts['CODI_DISTRICTE'] = df_districts['CODI_DISTRICTE'].astype(int)
df = df_water.merge(df_districts[['CODI_DISTRICTE', 'NOM_DISTRICTE']], left_on='District', right_on='CODI_DISTRICTE', how='left')
df.drop('CODI_DISTRICTE', axis=1, inplace=True)

ValueError: invalid literal for int() with base 10: '>'

# Visualizaciones del Consumo de Agua en Barcelona

En este notebook, exploraremos el consumo de agua acumulado diario en Barcelona, centrándonos en tendencias temporales, distribución por tipo de uso y por distrito.

In [ ]:
# Procesamiento de datos: Agregar consumo total por fecha y uso
df_grouped = df.groupby(['Date', 'Use'])['Accumulated_consumption_L_per_day'].sum().reset_index()

# Agregar consumo total por distrito
df_district = df.groupby(['NOM_DISTRICTE', 'Use'])['Accumulated_consumption_L_per_day'].sum().reset_index()

In [ ]:
# Visualización 1: Serie temporal del consumo acumulado por tipo de uso
fig1 = px.line(df_grouped, x='Date', y='Accumulated_consumption_L_per_day', color='Use', 
               title='Consumo Acumulado de Agua por Día y Tipo de Uso',
               labels={'Accumulated_consumption_L_per_day': 'Consumo Acumulado (L/día)', 'Date': 'Fecha'})
fig1.show()

In [ ]:
# Visualización 2: Gráfico de barras del consumo total por tipo de uso
total_by_use = df.groupby('Use')['Accumulated_consumption_L_per_day'].sum().reset_index()
fig2 = px.bar(total_by_use, x='Use', y='Accumulated_consumption_L_per_day', 
              title='Consumo Total de Agua por Tipo de Uso',
              labels={'Accumulated_consumption_L_per_day': 'Consumo Total (L)', 'Use': 'Tipo de Uso'})
fig2.show()

In [ ]:
# Visualización 3: Gráfico de barras apiladas del consumo por distrito y tipo de uso
fig3 = px.bar(df_district, x='NOM_DISTRICTE', y='Accumulated_consumption_L_per_day', color='Use', 
              title='Consumo de Agua por Distrito y Tipo de Uso',
              labels={'NOM_DISTRICTE': 'Distrito', 'Accumulated_consumption_L_per_day': 'Consumo (L)'})
fig3.show()

In [ ]:
# Visualización 4: Box plot del consumo diario por tipo de uso para detectar variabilidad
fig4 = px.box(df, x='Use', y='Accumulated_consumption_L_per_day', 
              title='Distribución del Consumo Diario por Tipo de Uso',
              labels={'Use': 'Tipo de Uso', 'Accumulated_consumption_L_per_day': 'Consumo (L/día)'})
fig4.show()